# Problem 3: Part B

In [72]:
# imports
import numpy as np
from sklearn.datasets import load_breast_cancer, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tqdm import tqdm
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler


In [73]:
def smooth_hinge_loss(z):
    return (np.maximum(0, 1 - z) ** 2)

def gradient_w(x, y, w, b, C):
    margin = y * (np.dot(w, x) + b)
    if margin >= 1:
        return np.zeros_like(w)
    else:
        return -2 * C * (1 - margin) * y * x

def gradient_b(x, y, w, b, C):
    margin = y * (np.dot(w, x) + b)
    if margin >= 1:
        return 0
    else:
        return -2 * C * (1 - margin) * y

def obj_func(X, y, w, b, C):
    hinge_losses = smooth_hinge_loss(y * (np.dot(X, w) + b))
    return 0.5 * np.dot(w, w) + C * np.sum(hinge_losses)

In [74]:
def gradient_descent(X, y, C, lr=0.1, max_iters=1000, tol=1e-5, beta=0.5, alpha=0.5):
    N, d = X.shape
    w = np.zeros(d)
    b = 0
    
    for iteration in range(max_iters):
        grad_w = np.zeros(d)
        grad_b = 0
        for i in range(N):
            grad_w += gradient_w(X[i], y[i], w, b, C)
            grad_b += gradient_b(X[i], y[i], w, b, C)
        
        grad_w += w
        
        # backtracking line search
        step_size = lr
        obj = obj_func(X, y, w, b, C)
        while obj_func(X, y, w - step_size * grad_w, b - step_size * grad_b, C) > obj - alpha * step_size * (np.dot(grad_w, grad_w) + grad_b ** 2):
            step_size *= beta
        
        # update parameters
        w -= step_size * grad_w
        b -= step_size * grad_b
        
        # check convergence
        if np.linalg.norm(grad_w) < tol and abs(grad_b) < tol:
            print(f"Converged in {iteration} iterations.")
            break
    
    return w, b


In [75]:
data = load_breast_cancer()
X = data.data
y = data.target
y[y == 0] = -1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C_values = [0.0001, 0.1, 1, 10, 100]

# run gradient descent using sklearn
for C in C_values:
    svc = SVC(C=C, kernel='linear')
    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"SVC with C={C} has accuracy: {accuracy:.4f}")

# run custom gradient descent
for C in C_values:
    w_opt, b_opt = gradient_descent(X_train, y_train, C)
    
    # Make predictions
    y_pred = np.sign(np.dot(X_test, w_opt) + b_opt)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Custom GD with C={C} has accuracy: {accuracy:.4f}")


SVC with C=0.0001 has accuracy: 0.9737
SVC with C=0.1 has accuracy: 0.9649
SVC with C=1 has accuracy: 0.9561
SVC with C=10 has accuracy: 0.9561
SVC with C=100 has accuracy: 0.9561
Custom GD with C=0.0001 has accuracy: 0.9649
Custom GD with C=0.1 has accuracy: 0.9649
Custom GD with C=1 has accuracy: 0.9649
Custom GD with C=10 has accuracy: 0.9649
Custom GD with C=100 has accuracy: 0.9649


# Problem 3: Part C

Mini-Batch Stoachastic Gradient Descent

In [76]:
def rbf_kernel(X, Z, gamma=0.05):
    X_norm = np.sum(X**2, axis=1).reshape(-1, 1)
    Z_norm = np.sum(Z**2, axis=1).reshape(1, -1)
    K = np.exp(-gamma * (X_norm + Z_norm - 2 * np.dot(X, Z.T)))
    return K

def smooth_hinge_loss(z):
    return (np.maximum(0, 1 - z) ** 2)

def smooth_hinge_loss_grad(z):
    return np.where(z < 1, -2 * (1 - z), 0)

def gradient_alpha(alpha, y, K, batch_indices, f_batch, C):
    N = len(batch_indices)
    grad_alpha = np.zeros(N)
    
    for k in range(N):
        # regularization term
        grad_alpha[i] = np.sum(alpha * y * K[i, :])

        # loss term
        for i, idx in enumerate(batch_indices):
            margin = y[idx] * f_batch[i]
            grad_alpha[k] += 2 * C / len(batch_indices) * smooth_hinge_loss_grad(margin) * y[idx] * y[k] * K[k, idx]

    return grad_alpha
        

def gradient_b(y, batch_indices, f_batch, C):
    grad_b = 0
    for i, idx in enumerate(batch_indices):
        margin = y[idx] * f_batch[i]
        grad_b += 2 * C / len(batch_indices) * smooth_hinge_loss_grad(margin) * y[idx]
        
    return grad_b


In [77]:
def load_mnist_data(samples=5000):
    # load MNIST
    mnist = fetch_openml('mnist_784')
    X = mnist.data.astype(np.float32)
    y = mnist.target.astype(np.int8)

    # convert to binary labels
    y_binary = np.where(y < 5, 1, -1)

    # reduce dataset size
    X, _, y_binary, _ = train_test_split(X, y_binary, train_size=samples, stratify=y_binary, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test
    
    

def stoacastic_gradient_descent_svm(X_train, y_train, C, lr=0.01, num_epochs=100, batch_size=32, gamma=0.05):
    N = X_train.shape[0]
    alpha = np.zeros(N)
    b = 0
    
    for epoch in tqdm(range(num_epochs)):
        # shuffle data
        perm = np.random.permutation(N)
        X_train = X_train[perm]
        y_train = y_train[perm]
        
        for i in range(0, N, batch_size):
            # mini-batch
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            
            K = rbf_kernel(X_train, X_batch, gamma=gamma)  # K(X_train, X_batch)
            f_batch = (alpha * y_train) @ K + b
            
            # squared hinge loss gradient
            loss_gradient = np.maximum(0, 1 - y_batch * f_batch)
            for j in range(len(X_batch)):
                if loss_gradient[j] > 0:
                    # update alpha
                    alpha[i+j] += lr * C * (1 - y_batch[j] * f_batch[j])
                    b += lr * C * y_batch[j]
            
        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{num_epochs}: Loss = {np.mean(loss_gradient)}")
    
    return alpha, b

def predict(X_train, y_train, alpha, b, X_test, gamma=0.05):
    return np.sign((alpha * y_train) @ rbf_kernel(X_train, X_test, gamma) + b)

In [78]:
def main():

    # hyperparameters
    C = 1
    batch_size = 64
    lr = 0.01
    num_epochs = 500
    gamma = 0.05

    # load data
    X_train, X_test, y_train, y_test = load_mnist_data()

    # standardize dataset
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # train model
    alpha, b = stoacastic_gradient_descent_svm(X_train, y_train, C=C, batch_size=batch_size, lr=lr, num_epochs=num_epochs, gamma=gamma)
    
    # evaluate model
    y_pred = predict(X_train, y_train, alpha, b, X_test, gamma=gamma)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    # evaluate against sklearn
    sgd_clf = SGDClassifier(loss='hinge', alpha=1/1, max_iter=1000, tol=1e-3)
    sgd_clf.fit(X_train, y_train)
    y_pred_sgd = sgd_clf.predict(X_test)
    accuracy_sgd = accuracy_score(y_test, y_pred_sgd)
    print(f"SGDClassifier Accuracy: {accuracy_sgd * 100:.2f}%")

main()

  0%|          | 2/500 [00:00<01:09,  7.16it/s]

Epoch 0/500: Loss = 1.0628766070919822


  2%|▏         | 12/500 [00:01<01:05,  7.46it/s]

Epoch 10/500: Loss = 0.882855023953566


  4%|▍         | 22/500 [00:03<01:10,  6.76it/s]

...

Epoch 480/500: Loss = 0.0


 98%|█████████▊| 492/500 [01:14<00:01,  6.77it/s]

Epoch 490/500: Loss = 0.0


100%|██████████| 500/500 [01:15<00:00,  6.63it/s]


Test Accuracy: 92.50%
SGDClassifier Accuracy: 84.30%
